# Plot Notebook

## Set up

In [36]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as pls
import matplotlib.dates as mdates
%matplotlib inline

from utils import read_stock

import plotly.express as px
import plotly.graph_objects as go


## Constants

In [37]:
PATH_STOCK = os.getcwd() + "/stocks/"

## Plot Bollinger Bands

In [38]:
def plot_bollinger_bands(df_stock: pd.DataFrame, ticker: str) -> None:
    """
    Function to plot Bollinger Bands.

    @param df_stock: df with the stats calculated
    @param ticker: Ticker of the stock to plot.
    @return: None
    """
    fig = go.Figure()

    candle = go.Candlestick(x=df_stock.index,
                            open=df_stock["Open"],
                            high=df_stock['High'], 
                            low=df_stock['Low'],
                            close=df_stock['Close'], 
                            name="Candlestick")

    upper_line = go.Scatter(x=df_stock.index,
                            y=df_stock['upper_band'],
                            line=dict(color='rgba(250, 0, 0, 0.75)',width=1),
                            name="Upper Band")
    
    mid_line = go.Scatter(x=df_stock.index,
                          y=df_stock['middle_band'],
                          line=dict(color='rgba(0, 0, 250, 0.75)', width=0.7),
                          name="Middle Band")

    lower_line = go.Scatter(x=df_stock.index,
                            y=df_stock['lower_band'], 
                            line=dict(color='rgba(0, 250, 0, 0.75)', width=1), 
                            name="Lower Band")

    fig.add_trace(candle)
    fig.add_trace(upper_line)
    fig.add_trace(mid_line)
    fig.add_trace(lower_line)

    fig.update_xaxes(title="Date", rangeslider_visible=True)
    fig.update_yaxes(title="Price")
    
    fig.update_layout(title=ticker + " Bollinger Bands",
    height=1200, width=1800, showlegend=True)
    fig.show()


In [39]:
ticker = "AAPL"
plot_bollinger_bands(read_stock(ticker, PATH_STOCK),ticker)

## Ichimoku

In [44]:
# Used to generate the red and green fill for the Ichimoku cloud
def get_fill_color(label):
    if label >= 1:
        return 'rgba(0,250,0,0.4)'
    else:
        return 'rgba(250,0,0,0.4)'

In [51]:
def plot_ichimoku(df_stock: pd.DataFrame) -> None:
    """
    Function to plot Ichimoku.

    @param df_stock: df with all the stats calculated.
    @return: None.
    """
    candle = go.Candlestick(x=df_stock.index, 
                            open=df_stock['Open'],
                            high=df_stock['High'],
                            low=df_stock["Low"],
                            close=df_stock['Close'],
                            name="Candlestick")

    df1 = df_stock.copy()
    fig = go.Figure()
    df_stock['label'] = np.where(df_stock['spanA'] > df_stock['spanB'], 1, 0)
    df_stock['group'] = df_stock['label'].ne(df_stock['label'].shift()).cumsum()

    df_stock = df_stock.groupby('group')

    dfs = []
    for name, data in df_stock:
        dfs.append(data)

    for df in dfs:
        fig.add_traces(go.Scatter(x=df.index, y=df.spanA,
        line=dict(color='rgba(0,0,0,0)')))

        fig.add_traces(go.Scatter(x=df.index, y=df.spanB,
        line=dict(color='rgba(0,0,0,0)'),
        fill='tonexty',
        fillcolor=get_fill_color(df['label'].iloc[0])))

    baseline = go.Scatter(x=df1.index, y=df1['baseline'], 
    line=dict(color='pink', width=2), name="Baseline")

    conversion = go.Scatter(x=df1.index, y=df1['conversion'], 
    line=dict(color='black', width=1), name="Conversion")

    lagging = go.Scatter(x=df1.index, y=df1['lagging'], 
    line=dict(color='purple', width=2), name="Lagging")

    span_a = go.Scatter(x=df1.index, y=df1['spanA'], 
    line=dict(color='green', width=2, dash='dot'), name="Span A")

    span_b = go.Scatter(x=df1.index, y=df1['spanB'], 
    line=dict(color='red', width=1, dash='dot'), name="Span B")

    fig.add_trace(candle)
    fig.add_trace(baseline)
    fig.add_trace(conversion)
    fig.add_trace(lagging)
    fig.add_trace(span_a)
    fig.add_trace(span_b)
    
    fig.update_layout(height=1200, width=1800, showlegend=True)

    fig.show()

In [53]:
ticker = "AAPL"
plot_ichimoku(read_stock(ticker, PATH_STOCK))